In [14]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components



Autosaving every 180 seconds


In [2]:
# Default functions
        
def load_csv(fname):
    with open(fname, newline='') as csvfile:
        data = list(csv.reader(csvfile))

    labels = data[1]
    #print ("data labels: ", labels)
    #print ("column vals: ", data[2])

    # load values
    data_array = np.array(data[3:])
    #print ("# of datapoints (x,y,likelihood): ", data_array.shape)

    # 
    #labels = ['fnose','f_leye','f_reye','f_lear','f_rear','f_',
    #         'male_nose','male_left_ear','male_right_ear','male_base_of_tail',
    #          'pup_shaved_nose','pup_shaved_left_ear','pup_shaved_right_ear','pup_shaved_base_of_tail',
    #          'pup_noshave_nose','pup_noshave_left_ear','pup_noshave_right_ear','pup_noshave_base_of_tail'             
    #         ]
    
    labels = labels[1:]
    
    traces = []
    traces_nan = []
    # zero out low quality DLC values
    for idx in range(1,len(labels)-1,3):
        #print ("idx: ", idx)
        #print (data_array[1:,idx:idx+3])
        #print (data_array[1:,idx:idx+3].shape)

        temp = data_array[1:,idx:idx+3]
        idx1 = np.where(temp=='')[0]
        temp[idx1]=0
        temp = temp.astype(np.float)# np.array(temp)
        #print (idx, "TEMP: ", temp.shape)
        #temp.replace("''",'0')
        

        # replace low likelihoods with median
        likelihoods = temp[:,2]
        idx2 = np.where(likelihoods<0.8)[0]
        temp[idx2,0]=np.median(temp[:,0])
        temp[idx2,1]=np.median(temp[:,1])
        traces.append(temp.copy())
        
        temp[idx2,0]=np.nan
        temp[idx2,1]=np.nan
        traces_nan.append(temp.copy())

    return traces, labels, traces_nan

# function that does search forward steps: 
def search_forward(data_assembled_fixed,
                  traces_inf,
                  threshold_p,
                  dist_threshold,
                  data_assembled_all_features,
                  selected_feature,
                  comments=False):

    # count # of merges
    n_merges = 0
    
    # load traces
    tracex = data_assembled_fixed[:,0]
    tracey = data_assembled_fixed[:,1]

    # find assembled locations with high prob. 
    probs = data_assembled_fixed[:,2]
    idx = np.where(probs>threshold_p)[0]
    
    # if no values over threshold skip chunk;
    if idx.shape[0]==0:
        return data_assembled_fixed, data_assembled_all_features
    
    
    if comments:
        print ("data assembled fixed: ", data_assembled_fixed.shape)
        print ("traces inf: ", traces_inf.shape)
        print ("probs: ", probs[:10])
        print ("IDX: ", idx.shape)

    # find ends of continous labeled segments
    # old method
    #diffs = idx[1:]-idx[:-1]
    #seg_ends = np.where(diffs>1)[0]
    seg_ends=[]
    for k in range(idx.shape[0]-1):
        if (idx[k+1]-idx[k])>1:
            seg_ends.append(k)
    # append the last value if it's not at the end of the data
    if idx.shape[0]<data_assembled_fixed.shape[0]:
        if idx.shape[0]>0:
            seg_ends.append(idx.shape[0]-1)
    #print ("idx shape:" , idx.shape[0], " tracex: ", tracesx.shape[0])
    #print ("Actual last idx values: ", idx[-1])

    seg_ends = np.array(seg_ends)
    #print (seg_ends.shape, idx[-1].shape, idx[-1])
    # add last segment also
    #if idx[-1]!=(tracex.shape[0]-1):
    #    seg_ends.append()
#     if idx.shape[0]>0:
#         seg_ends=np.concatenate((seg_ends, [idx[-1]]),axis=0)
    
    if comments:
        #print ("diffs[ends]: ", diffs[seg_ends])
        if seg_ends.shape[0]>0:
            print (seg_ends)
            print ("seg ends: ", idx[seg_ends])

    # propagate forward
    # loop over all ends of continous segments and search forward for min distances
    for l in range(seg_ends.shape[0]):
        idx_start = idx[seg_ends[l]]

        # initalized with the first starting location;
        loc0 = np.array([tracex[idx_start], tracey[idx_start]])

        # search forward
        idx_next = idx_start+1
        if idx_next>=1000:
            continue
        while True:
            if comments:
                print ("idx_current: ", idx_next-1 , "/", seg_ends.shape[0],
                       "loc0: ", loc0, " prob: ", probs[idx_next-1])
                print ("idx_next: ", idx_next , 
                       "loc0: ", np.array([tracex[idx_next], tracey[idx_next]]), " prob: ", probs[idx_next])

            # if there is no inference, delete data
            if len(traces_inf[idx_next])==0:
                break
                
            loc_candidates = np.vstack(traces_inf[idx_next]).T
            if comments:
                print ("loc_candidates: ", loc_candidates)
  
            # compute distance between previous assembled (true) location and canadidate
            dist = (loc_candidates.T - loc0)**2
            dist = np.sum(dist, axis=1)
            dist = np.sqrt(dist)

            # minimum distance is less than threshold, add to assembled animal data;
            min_dist = np.min(dist)
            arg_min = np.argmin(dist)
            if comments:
                print ("Dist: ", min_dist, " argmin: ", arg_min, " values; ", loc_candidates.T[arg_min])

            if min_dist < dist_threshold:
                
                # first check if the value has alerady been assigned to an assembled animal;
                if comments:
                    print (" CROSS VALUE SEARCH *****************************************************")
                    print (data_assembled_all_features.shape)
                    print ("data assembled all features; ", data_assembled_all_features[:,idx_next,0])
                    print ("best inference candidate: ", loc_candidates[0][arg_min])
                    print (np.min(np.abs(data_assembled_all_features[:,idx_next, 0]-loc_candidates[0][arg_min])))

                # Chekc if x values are identical between a previously assembled feature and the best inference match
                # if so, check if the probability of the assembled feature is above thrshold (ie. >0 as it's already been set to 0)
                # and skip it; 
                # otherwise, do not inherit the label and exit the segment
                if np.min(np.abs(data_assembled_all_features[:,idx_next,0]-loc_candidates[0][arg_min]))<1E-5:
                    if np.min(np.abs(data_assembled_all_features[:,idx_next,1]-loc_candidates[1][arg_min]))<1E-5:
                        argmin_temp = np.argmin(np.abs(data_assembled_all_features[:,idx_next,0]-
                                                    loc_candidates[0][arg_min]))
                        if data_assembled_all_features[argmin_temp,idx_next,2]>0.0:
                            if comments:
                                print ("Infered value belongs to already assbmeld feature", argmin_temp,
                                      "  with prob: ", data_assembled_all_features[argmin_temp,idx_next,2])
                                print ("data_assembled_all_features[idx_next]: ", data_assembled_all_features.shape)
                                print (np.argmin(np.abs(data_assembled_all_features[:,0]-loc_candidates[0][arg_min])))

                            break

                if comments:
                    print ("replace assembled val at time step: ", idx_next, " at loc: ", loc0 )
                    
                loc0 = np.array([loc_candidates[0][arg_min],
                                loc_candidates[1][arg_min]])
                if comments:

                    print ("      with new location from inference", loc0)

                    print ('data_assembled_fixed[feature][pre]: ', 
                           data_assembled_fixed[idx_next])

                
                # fix the data in progress
                data_assembled_fixed[idx_next]=np.array([loc0[0],  # set x
                                                         loc0[1],  # set y
                                                         1.0])     # set probability
                # fix master list as well
                data_assembled_all_features[selected_feature,idx_next]=np.array([loc0[0],  # set x
                                                         loc0[1],  # set y
                                                         1.0])     # set probability
                
                # metadata printing
                if comments:
                    print ('data_assembled_fixed[feature][post]: ', 
                       data_assembled_fixed[idx_next])

                    print ('')
                    
                    
                idx_next+=1
                n_merges+=1
                # exit if at end of data
                if idx_next>=1000:
                    break

                # exit if reached a chunk that is labled above accepted probability:
                if data_assembled_fixed[idx_next][2]>threshold_p:
                    if comments:
                        print ("***** Point has prob > threshold (moving to next discontious segment) *****")
                        print ("")
                        print ("")
                        print ("")
                    break


            else: #move to the next continous segment if distance to nearest time point is too far
                if comments:
                    print ("***** JUMPING OT NEXT SEG *****")
                    print ("")
                    print ("")
                    print ("")
                break

            #return
                
    print ('********** # OF MERGES: ', n_merges)

    # return fixed data
    return data_assembled_fixed, data_assembled_all_features

In [24]:
fnames = [
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_54_23_358257_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02_54_39_170978_compressed/2020-3-16_02_54_39_170978_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03_57_56_902379_compressed/2020-3-16_03_57_56_902379_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03 54 54 231226_compressed/2020-3-16_03 54 54 231226_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_55_09_841582_compressed/2020-3-16_04_55_09_841582_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_58_11_998956_compressed/2020-3-16_04_58_11_998956_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_05 55 25 305681_compressed/2020-3-16_05 55 25 305681_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_05 58 27 193818_compressed/2020-3-16_05 58 27 193818_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_06 55 40 714236_compressed/2020-3-16_06 55 40 714236_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_06 58 43 678014_compressed/2020-3-16_06 58 43 678014_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_55_55_775234_compressed/2020-3-16_07_55_55_775234_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_59_00_362242_compressed/2020-3-16_07_59_00_362242_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_08_56_11_096689_compressed/2020-3-16_08_56_11_096689_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_08_59_17_534732_compressed/2020-3-16_08_59_17_534732_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09_59_34_731308_compressed/2020-3-16_09_59_34_731308_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09 56 26 362091_compressed/2020-3-16_09 56 26 362091_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_10_59_50_448686_compressed/2020-3-16_10_59_50_448686_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_10 56 41 406701_compressed/2020-3-16_10 56 41 406701_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_11_56_56_704655_compressed/2020-3-16_11_56_56_704655_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_12 57 12 418305_compressed/2020-3-16_12 57 12 418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02 57 41 995158_compressed/2020-3-16_02 57 41 995158_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle'
         ]

In [19]:
# LOAD INFERENCE DATA from full_pickle files and convert to  .npy file
#full_pickle= '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle'

for full_pickle in fnames:
    print ("processing :", full_pickle)
    # convert full pickel file to simpler data structure
    fname_out = full_pickle[:-7]+"_traces_inferences.npy"
    if os.path.exists(fname_out)==False:
        from deeplabcut.pose_estimation_tensorflow.lib.inferenceutils import (
                convertdetectiondict2listoflist)
        import pickle, re
        # load pickle and 
        with open(full_pickle, "rb") as file:
            data = pickle.load(file)
        header = data.pop("metadata")
        all_jointnames = header["all_joints_names"]

        #if displayedbodyparts == "all":
        if True:
            numjoints = len(all_jointnames)
            bpts = range(numjoints)

        frame_names = list(data)
        frames = [int(re.findall(r"\d+", name)[0]) for name in frame_names]

        # Convert inference locations to an easier array to parse

        traces_inferences = []
        ctr=0
        start = 0
        end = len(frame_names)

        for n in trange(start, end, 1):
            #ind = n
            #print (n, frame_names[n])
            #print (data)
            # load inference locations
            traces_inferences.append([])
            dets = convertdetectiondict2listoflist(data[frame_names[n]], bpts)
            for i, det in enumerate(dets):
                traces_inferences[ctr].append([])
                for x, y, p, _ in det:
                    traces_inferences[ctr][i].append([x,y])

            ctr+=1
        traces_inferences = np.array(traces_inferences)
        print (traces_inferences.shape)

        np.save(fname_out, traces_inferences)

    else:
        traces_inferences = np.load(fname_out, allow_pickle=True)
    print (" traces_inferences [n_frames, #_non_unique_featres]: ", traces_inferences.shape)


processing : /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02 57 41 995158_compressed/2020-3-16_02 57 41 995158_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle


100%|██████████| 89989/89989 [00:19<00:00, 4591.47it/s]


(89989, 14)
 traces_inferences [n_frames, #_non_unique_featres]:  (89989, 14)
processing : /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle


100%|██████████| 89988/89988 [00:20<00:00, 4377.67it/s]


(89988, 14)
 traces_inferences [n_frames, #_non_unique_featres]:  (89988, 14)


In [15]:
fname_out = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/traces_inferences.npy'
traces_inferences = np.load(fname_out, allow_pickle=True)
print (" traces_inferences [n_frames, #_non_unique_featres]: ", traces_inferences.shape)



 traces_inferences [n_frames, #_non_unique_featres]:  (89989, 14)


In [16]:
# fnames = ['/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02 57 41 995158_compressed/2020-3-16_02 57 41 995158_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
#          '/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle'
#          ]

In [26]:
# #########################################################
# #########################################################
# ############## CONNECTED COMPONENTS SOLUTION ############
# #########################################################
# #########################################################

#for fname in fnames:
def connected_components_function(fname_in):
    fname_in = fname_in[:-7]+"_traces_inferences.npy"
    traces_inferences = np.load(fname_in, allow_pickle=True)
    print (" traces_inferences [n_frames, #_non_unique_featres]: ", traces_inferences.shape)
    print (fname_in)

    fname_out = fname_in[:-4]+'.npz'

    #[start:end,feature]
    labels = []
    #kmeans_start = 54000
    #kmeans_end = kmeans_start+1000
    kmeans_start = 0
    kmeans_end = traces_inferences.shape[0]


    # ##########################################################
    # ################### PARAMETER LISTS ######################
    # ##########################################################
    max_dist_between_features = 100 # connected componennts max distances allowed between sequential points
    min_network = 5  # min length of segs to connect
    min_dist_chunks = 200 # min distance between matching history
    n_networks = 4 # essentially Number of animals that are matched to a graph
    network_history = 30
    max_network_jump = 300 # maximum amount an animal id (centre) can move in a single frame 
    max_dist_merge_networks_cc = 75 # post-cc step to fix oversplits; searches cnetres of two networks
                                     # with non-overlapping features and merges them


    # otehr params
    min_n_matches_history_override = 20 # this parameter indicates if 5 feature-wise
                                       # very close matches in previous history
                                       # will override the centres-based history tracker


    # ##########################################################
    # ##########################################################
    # ##########################################################

    comments = False

    labels_array = []
    features_array = []
    ctr=0
    historical_centres = np.zeros((network_history,n_networks,2))
    # loop over frames
    ctr_comments =[]
    for k in range(kmeans_start, kmeans_end):

        if ctr in ctr_comments:
            comments=True
        else:
            comments=False

        if True:
            if k%10000==0:
                print ("FRAME: ", k)

        # make label array
        labels_array.append([])
        features_array.append([])

        # LOAD DATA FROM INFERENCE STEP
        flat_list = []
        list_idx = []
        for n, sublist in enumerate(traces_inferences[k]):
            list_idx.extend(np.zeros(len(sublist),'int32')+n)
            for item in sublist:
                flat_list.append(item)

        #print (flat_list)
        locs = np.vstack(flat_list).copy()
        list_idx = np.array(list_idx)


        # ####################################################################################
        # ####################################################################################
        # ########################### CONNECTED COMPONENTS ###################################
        # ####################################################################################
        # ####################################################################################
        # loop over features and build graphs:
        # find nearest next feature to the selected feature and connect them:
        cc = []
        cc2 = np.zeros((list_idx.shape[0],list_idx.shape[0]),'int32')

        # loop over each group of features in the dataset 
        for feature in range(np.unique(list_idx).shape[0]-1):
            idx = np.where(list_idx==feature)[0]
            locs_feat_current = locs[idx]

            # grab all features in the next step of hierarchy and mathc to previous hierarchy features;
            for qq in range(feature+1,feature+2):
                idx_next = np.where(list_idx==qq)[0]
                locs_next = locs[idx_next]

                if locs_next.shape[0]==0:
                    continue

                # Compute shortest distance between the current features and the next step features
                dists = []
                argmins = []
                for p in range(locs_feat_current.shape[0]):
                    loc_ = locs_feat_current[p]

                    vect = locs_next-loc_
                    min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                    argmin = np.argmin(scipy.spatial.distance.cdist(vect*0, vect))
                    dists.append(min_dist)
                    argmins.append(argmin)

                # ranked by distance
                dists = np.array(dists)
                argmins = np.array(argmins)
                idx_sort = np.argsort(dists)

                for p in range(idx_sort.shape[0]):
                    # load sorted data
                    min_dist = dists[idx_sort[p]]
                    argmin = argmins[idx_sort[p]]

                    if min_dist<max_dist_between_features:
                        # check to ensure the same type of feature doesn't already exist in the dataset:
                        # actually need to check if any of the other parts are connected, not just the previous most recent part?! 
                        # idx here is all the locations of current hierarchy features
                        if np.any(cc2[idx,idx_next[argmin]]):
                            if comments: 
                                print("feature already connected to other parts; skipping")
                            continue
                        #cc.append([idx[p], idx_next[argmin]])
                        cc2[idx[idx_sort[p]], idx_next[argmin]]=1


        graph=np.array(cc2)
        graph = csr_matrix(graph)


        n_components, labels_all = connected_components(csgraph=graph, directed=False, return_labels=True)
        #print (n_components)

        # find all unique network ids; labels_unique[0] - ids;  labels_unique[1] is counts
        labels_unique = np.unique(labels_all, return_counts=True)

        # keep only networks over min_network size;
        idx_nets = np.where(labels_unique[1]>=min_network)[0]
        labels_kept = labels_unique[0][idx_nets]

        # order the networks by size and keep only to n_networks
        idx_ordered = np.argsort(labels_unique[1][idx_nets])[::-1]
        labels_kept = labels_kept[idx_ordered][:n_networks]
        if comments:
            print ("idx_networks kept:", idx_nets)
            print ("labels kept: ", labels_kept)
            print (" size of networks kept: ", labels_unique[1][idx_nets][idx_ordered][:n_networks])

            for p in np.unique(labels_kept):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]
                print ("network :", p, locs_network)
                print ('size of network ', locs_network.shape)
                print (" feature types; ", list_idx[idx])
            print ("")

        # final cc check for oversplit clusters:
        # note: This should be done in the CC matrix step, but can't seem to get it to work in full
        labels_kept_temp = labels_kept.copy()
        for p in np.unique(labels_kept):
            idx1 = np.where(labels_all==p)[0]
            features1 = list_idx[idx1]
            locs_network1 = locs[idx1]

            if idx1.shape[0]==0:
                continue

            for pp in np.unique(labels_kept):
                if pp==p:
                    continue
                idx2 = np.where(labels_all==pp)[0]
                if idx2.shape[0]==0:
                    continue

                features2 = list_idx[idx2]
                locs_network2 = locs[idx2]

                # check if nonoveralpping features
                if np.any(np.isin(features1,features2))==False:

                    # chekc if very close in distance
    #                 vect = labels_array[ctr-1][z]-locs_feature
    #                 min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                    dists = scipy.spatial.distance.cdist(locs_network1, locs_network2)
                    if comments:
                        print ("dists between nonvoeralpping networks: ", dists)

                    # merge bits
                    if np.min(dists)<max_dist_merge_networks_cc:

                        # change labels_all and labels_kept
                        labels_all[idx2] = p
                        idx_del = np.where(labels_kept_temp==pp)[0]
                        labels_kept_temp = np.delete(labels_kept_temp, idx_del)

                        if comments:
                            print ("merged networks: ", p, " and ", pp, " at idx: ", idx_del)


        labels_kept = labels_kept_temp
        #print ("LABELS post cc: ", labels)

        # ####################################################################################
        # ####################################################################################
        # ########################### FRAME LOOP ANALYSIS ####################################
        # ####################################################################################
        # ####################################################################################
        # save previous centres to match them and overwrite below
        # loop over all network components
        if ctr>0:
            # make large list to hold ftuure labels
    #        features_list = [
            for p in range(n_networks):
                labels_array[ctr].append([])
                features_array[ctr].append([])

            # keep track of the matches with previous chunks
            previous_centres = historical_centres.copy() #[0,p]=np.mean(locs_network,0))

            if comments:
                print ("previous centres: ", previous_centres.shape)
            #break

            idx_matches = []
            idx_matches_p = []


            # ####################################################################################
            # ########################### COMPUTE DISTANCES TO PREVIOUS FRAME ####################
            # ####################################################################################

            # search for current network chunk for nearest previous chunk
            # count the number of matches between current network centre and history:
            n_matches_history = np.zeros((labels_kept.shape[0], previous_centres.shape[1]),'int32')
            total_dist = np.zeros((labels_kept.shape[0],previous_centres.shape[1]),'float32') 

            # new method checks averages
            # and also whether min_network closest matches to the previous immediate vals;
            # If option 2 is very close it overrides all other matches. 
            for ctr_p, p in enumerate(np.unique(labels_kept)):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]

                # compute cntre:
                centre_network = np.median(locs_network,0)
                if comments:
                    print ("centre network", p, centre_network)
                    print (" doing feature-wise heuristic search at prev step: ")
                    #print ("      Previous time step data", labels_array[ctr-1])
                    #print ("      current network: ", locs_network)
                # also quick check if the previous steps had values very close to current 
                # loop over current network values and find shortest 5 distances:
                min_dist_count = np.zeros(n_networks, 'int32')
                for zz in range(locs_network.shape[0]):
                    locs_feature = locs_network[zz]
                    if comments:
                        print ("search for close feature to ", locs_feature)
                        #if comments:

                    #min_dist_count = 0
                    for z in range(len(labels_array[ctr-1])):
                        temp_temp = labels_array[ctr-1][z]
                        if len(labels_array[ctr-1][z])==0:
                            continue
                        vect = labels_array[ctr-1][z]-locs_feature
                        min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                        #print ("min dist: ", min_dist)
                        if min_dist < min_n_matches_history_override:
                            min_dist_count[z]+=1

                # if sufficient matches no need to compute and check centres
                max_nearest = np.max(min_dist_count)
                if max_nearest > 5:
                    argmax_overwrite = np.argmax(min_dist_count)
                    #n_matches_history[:,argmax_overwrite]-=100  # ensure the previous neetwork can only be matched to the current one
                                                      # by starting/biasing the match count for the rest of the matches;
                                                      # can also do this other ways
                    n_matches_history[ctr_p,argmax_overwrite]+=1E5
                    total_dist[ctr_p,argmax_overwrite]=0 #previous_c

                    if comments:
                        print ("nearly similar network at previous time step; ")
                        print (" Min dist array: ", min_dist_count)
                        print ("         matching to prev network using argmax: ", argmax_overwrite)
                        print ("         current locs ", locs_network)
                        print ("         previous matched network: ", labels_array[ctr-1][argmax_overwrite])

                else:
                    # loop over each previous network
                    for z in range(previous_centres.shape[1]):

                        # loop over each previous history point: find how many historical matches it has
                        for h in range(previous_centres[:,z].shape[0]):

                            # May wish to implement an L1 not L2 distance 
                            # OR an individual feature distance rather than mean
                            # OR find median xy point and use that as centre;
                            #if True:
                            vect = previous_centres[h,z]-centre_network
                            min_dist = np.min(np.linalg.norm(vect))
                            #print ("VECT: ", vect, " Min dist: ", min_dist)
                            #min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                            if min_dist<min_dist_chunks:
                                n_matches_history[ctr_p][z]+=1
                                total_dist[ctr_p][z]+=min_dist #previous_centres[:,z].shape[0]-h)/previous_centres[:,z].shape[0])
                                #total_dist[z]+=min_dist*((previous_centres[:,z].shape[0]-h)/previous_centres[:,z].shape[0])

                                if comments:
                                    print (z,h,"adding dist: ", min_dist)

            # check best matches from history
            for ctr_p, p in enumerate(np.unique(labels_kept)):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]

                if comments:
                    print ("")
                    print ("animal:", ctr_p, "n_mathces_history:", n_matches_history[ctr_p])
                    print ("locs: ", locs_network)

                # Loop to ensure no better match found for a previous animal ID        
                while True:
                    argmax = np.argmax(n_matches_history[ctr_p])
                    if comments:
                        print ("curretn best match for current locs network: ", locs_network)
                        print ("   is prev network: ", labels_array[ctr-1][argmax])

                    if n_matches_history[ctr_p,argmax]==0:
                        break

                    # ensure no boetter match was found for the current val
                    n_matches_all_column = n_matches_history[:,argmax]
                    if np.max(n_matches_all_column)>n_matches_history[ctr_p,argmax]:

                        argmax_col = np.argmax(n_matches_all_column)

                        # check also distances for better fit:
                        current_dist = total_dist[ctr_p][argmax]/n_matches_history[ctr_p][argmax]
                        alternative_match_dist = total_dist[argmax_col][argmax]/n_matches_history[argmax_col][argmax]

                        if comments:
                            print (" current dist: ", current_dist)
                            print (" alternative dist: ", alternative_match_dist)

                        if alternative_match_dist< current_dist:

                            n_matches_history[ctr_p, argmax]=0
                            if comments: 
                                print (" FOUND BETTER MACH WITH ANOTHER ANIMAL, setting argmax to zoer")
                        else:
                            break
                    else:
                        break


                # check case when more than 1 animal within merging distance and pick closest
                max_count = n_matches_history[ctr_p][argmax]
                if comments:
                    print (" argmax in match history: ", argmax)
                    print ('               max count: ', max_count)

                # check if more than one match for new network
                # may wish to take all matches > 15 or max

                idx11 = np.where(n_matches_history[ctr_p]==max_count)[0]
                if (idx11.shape[0]>1) and max_count>0:

                    # find what the averate distance between each matched history and the new frame are and 
                    # take the shortest average distance
                    # check all the matches that had same amount as idx11
                    match_means = total_dist[ctr_p][idx11]/n_matches_history[ctr_p][idx11]
                    argmin_match = np.argmin(match_means)
                    argmax = idx11[argmin_match]
                    min_dist_found = match_means[argmin_match]

                    if comments:
                        print ("DUPLICATE MATCHES: so looking at distances: ", total_dist[ctr_p][idx11])
                        print ("Original min distance found: ", min_dist_found)
                        print ("Average distancesL", match_means, " of argmin: ", argmin_match)
                        print ("total dists: ", total_dist[ctr_p])

                    # check if matched network doesn't actually have even closer match:
                    # so search in the column of the original best match to see if it has a better match
                    idx13 = np.where(n_matches_history[:,argmax]==max_count)[0]
                    match_means_network_match = total_dist[:,argmax][idx13]/n_matches_history[:,argmax][idx13]
                    argmin_match_near = np.argmin(match_means_network_match)
                    if match_means_network_match[argmin_match_near]<min_dist_found:

                        # set the distance for the previous best match to very large value
                        total_dist[ctr_p][argmax] = 1E10

                        # recompute the argmax    
                        match_means = total_dist[ctr_p][idx11]/n_matches_history[ctr_p][idx11]
                        argmin_match = np.argmin(match_means)
                        argmax = idx11[argmin_match]
                        min_dist_found = match_means[argmin_match]

                        if comments:
                            print ("BETTER MATCH WAS FOUND", argmin_match)
                            print ("new min distance found: ", min_dist_found)


                        #else:
                        #    break
                    else:
                        if comments:
                            print ("NO BETTER MATCH FOUND")

                # check also to see if networked matched on centre-based distance doesn't actually ahve a better match via
                #   feature-based matching with another dataset;
                n_matches_all_column = n_matches_history[:,argmax]
                if np.max(n_matches_all_column)>n_matches_history[ctr_p,argmax]:
                    if comments:
                        print ("  Matches in columN: ", n_matches_all_column)
                        print ("     ARE GREATER THAN BEST Centre based match: ", n_matches_history)
                        print ("  ***************************************")

                # ############################################################ 
                # ######################## SAVE MATCH DATA ################### 
                # ############################################################ 
                if n_matches_history[ctr_p][argmax]>0:
                    # replace locations of previous match 
                    idx_matches.append(argmax)
                    idx_matches_p.append(p)
                    if comments:
                        print ("Matched to previous centre: ", argmax)
                        print ("final argmax: ", argmax)
                        print ("locs current network:", locs_network)
                        print ("to be inserted near previous network: ", labels_array[ctr-1][argmax])

                # add it as a new tracklet
                else:
                    idx_matches.append(None)
                    idx_matches_p.append(p)

                # zero out the better match in this column
                total_dist[:,argmax]=1E10
                n_matches_history[:,argmax] = 0

            # ############################################################
            # ############################################################
            # ############### INSERT THE MATCHED DATA ####################
            # ############################################################
            # ############################################################
            if comments:
                print("")
                print("FINAL NETWORK INSERTION STEP")

            # loop over the matched ids
            for p in range(len(idx_matches)):
                idx = np.where(labels_all==idx_matches_p[p])[0]
                locs_network = locs[idx]
                feature_types = list_idx[idx]
                if comments:
                    print ("idx_matches[p]:", idx_matches[p])

                if idx_matches[p] is not None:

                    # do a final check to make sure neighbouring timestep networks aren't super-far apart
                    centre_prev_network = np.median(labels_array[ctr-1][idx_matches[p]],0)
                    centre_current_network = np.median(locs_network,0)
                    vect = centre_prev_network-centre_current_network
                    dist_centre = np.linalg.norm(vect)
                    if comments:
                        print ("   FINAL CHECK DISTANCE BETWEEN PREVIOUS AND CURRENT CENTRE",
                              "  vect" , vect,
                              "  dist: ", dist_centre)
                    if dist_centre>=max_network_jump:
                        if comments:
                            print (" SKIPPED NETWORK DUE TO LARGE JUMP!")
                    else:
                        labels_array[ctr][idx_matches[p]]=locs_network
                        features_array[ctr][idx_matches[p]]=feature_types

                        if comments:
                            print ("")
                            print ("inserted: ", locs_network)
                            print (" at location: ", idx_matches[p])
                            print ("ctr-1 values at this location: ",idx_matches[p],
                                   " has values: ",
                                   labels_array[ctr-1][idx_matches[p]])
                else:
                    if comments:
                        print ("network doesn't have match: ", locs_network)

            # LABELS FOR UNMATCHED DATA
            ctr_inner=0
            for p in range(len(idx_matches)):
                if idx_matches[p] is None:
                    idx = np.where(labels_all==idx_matches_p[p])[0]
                    feature_types = list_idx[idx]
                    locs_network = locs[idx]
                    if comments:
                        print ("Inserting missed network to end:", locs_network)
                    # search for an empty list to populate
                    while True:
                        # find the first empty location to insert data
                        if len(labels_array[ctr][ctr_inner])==0:

                            # do a final check to make sure neighbouring timestep networks aren't super-far apart
                            centre_prev_network = np.median(labels_array[ctr-1][ctr_inner],0)
                            centre_current_network = np.median(locs_network,0)
                            vect = centre_prev_network-centre_current_network
                            dist_centre = np.linalg.norm(vect)
                            if comments:
                                print ("   FINAL CHECK DISTANCE BETWEEN PREVIOUS AND CURRENT CENTRE",
                                      "  vect" , vect,
                                      "  dist: ", dist_centre)
                            if dist_centre>=max_network_jump:
                                if comments:
                                    print (" SKIPPED NETWORK DUE TO LARGE JUMP!")

                                # exit and skip this segment altogether
                                break

                            else:
                                labels_array[ctr][ctr_inner]=locs_network
                                features_array[ctr][ctr_inner]=feature_types
                                ctr_inner+=1
                                break
                        ctr_inner+=1

            # add centres to 

            historical_centres[1:]=historical_centres[:-1] #[0,p]=np.mean(locs_network,0))
            for pp in range(len(labels_array[ctr])):
                if len(labels_array[ctr][pp])==0:
                    continue
                locs_new = labels_array[ctr][pp]
                centres_new = np.median(locs_new,0)
                if comments:
                    print ("centres new", pp, centres_new)
                #historical_centres[0,pp]=centres_new
                historical_centres[0,pp]=centres_new


            if comments:
                print (k, " FINAL labels array: ", labels_array[ctr])

        # for first time point
        else:
            for k in range(n_networks):
                labels_array[ctr].append([])
                features_array[ctr].append([])

            for n, p in enumerate(np.unique(labels_kept)):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]
                feature_types = list_idx[idx]

                #print ("locs network", locs_network)
                labels_array[ctr][n]=locs_network
                features_array[ctr][n]=feature_types

                # fill the entire history with starting locations:
                historical_centres[:,n]=np.median(locs_network,0)

            if comments:
                print ("FIRST saved labels array: ", labels_array)
    #             break
    #         break
    #     break

        ctr+=1   
        #print ("")
        #print ("")

    # convert the output to a rectangular matrix with correct locations for data;
    final_features = np.zeros((56, len(labels_array),2), 'float32')
    print ("Final eatures; ", final_features.shape)
    for k in range(len(labels_array)):
        if k%1000==0:
            print (k)
        for p in range(len(labels_array[k])):
            #for l in range(len(labels_array[k][p])):
                #print (data[k][p][l])
            label_locs = features_array[k][p]
            if len(label_locs)>0:
                #print (k, p, label_locs)
                final_features[label_locs+p*14,k]=labels_array[k][p]

    tracesx = final_features[:,:,0]
    tracesy = final_features[:,:,1]
    probs = np.ones(tracesy.shape, 'float32')
    print ("SAVING:to file: ", fname_out)
    np.savez(fname_out,
            tracesx=tracesx,
            tracesy=tracesy,
            probs =probs
            )    

if True:
    import parmap
    parmap.map(connected_components_function, fnames,
              pm_processes=12)

 traces_inferences [n_frames, #_non_unique_featres]:  (89989, 14)
/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03_57_56_902379_compressed/2020-3-16_03_57_56_902379_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npy
FRAME:  0
 traces_inferences [n_frames, #_non_unique_featres]:  (89989, 14)
/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_58_11_998956_compressed/2020-3-16_04_58_11_998956_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npy
FRAME:  0
 traces_inferences [n_frames, #_non_unique_featres]:  (89988, 14)
/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_59_00_362242_compressed/2020-3-16_07_59_00_362242_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npy
FRAME:  0
 traces_inferences [n_frames, #_non_unique_featres]:  (89988, 14)
/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/ma

39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
SAVING:to file:  /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_58_11_998956_compressed/2020-3-16_04_58_11_998956_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz
FRAME:  80000
 traces_inferences [n_frames, #_non_unique_featres]:  (89986, 14)
/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09 56 26 362091_compressed/2020-3-16_09 56 26 362091_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npy
FRAME:  0
FRAME:  10000
FRAME:  80000
FRAME:  70000
FRAME:  80000
FRAME:  60000
FRAME:  80000
FRAME:  10000
FRAME:  10000
Final eatures;  (56, 89988, 2)
0
1000
2000
30

87000
88000
89000
SAVING:to file:  /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_55_55_775234_compressed/2020-3-16_07_55_55_775234_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz
FRAME:  40000
FRAME:  20000
 traces_inferences [n_frames, #_non_unique_featres]:  (89988, 14)
/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npy
FRAME:  0
FRAME:  20000
FRAME:  30000
FRAME:  10000
FRAME:  80000
FRAME:  30000
FRAME:  30000
FRAME:  10000
FRAME:  30000
FRAME:  40000
FRAME:  20000
FRAME:  30000
FRAME:  30000
Final eatures;  (56, 89988, 2)
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000

53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
SAVING:to file:  /mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09_59_34_731308_compressed/2020-3-16_09_59_34_731308_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz
FRAME:  80000
FRAME:  70000
Final eatures;  (56, 89986, 2)
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83

In [335]:
# convert the output to a rectangular matrix with correct locations for data;
final_features = np.zeros((56, len(labels_array),2), 'float32')
print ("Final eatures; ", final_features.shape)
for k in range(len(labels_array)):
    if k%1000==0:
        print (k)
    for p in range(len(labels_array[k])):
        #for l in range(len(labels_array[k][p])):
            #print (data[k][p][l])
        label_locs = features_array[k][p]
        if len(label_locs)>0:
            #print (k, p, label_locs)
            final_features[label_locs+p*14,k]=labels_array[k][p]
            
tracesx = final_features[:,:,0]
tracesy = final_features[:,:,1]
probs = np.ones(tracesy.shape, 'float32')
np.savez('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/labels_array.npz',
        tracesx=tracesx,
        tracesy=tracesy,
        probs =probs
        )

Final eatures;  (56, 89500, 2)
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000


In [341]:
idx_nan = np.where(final_features==0)
#print (idx_nan)
temp_features = final_features.copy()

temp_features[idx_nan]=np.nan

fig=plt.figure()
print (final_features.shape)
for p in range(4):
    #print (feature+p*14)
    #idx = np.where(final_features[feature+p*14,:,0]!=0)[0]
    
    locs = np.nanmean(np.nanmean(temp_features[p*14: (p+1)*14,:,:],2),0)
    print (locs.shape)
    plt.scatter(np.arange(locs.shape[0]), locs,c=clrs[p])
plt.show()

(56, 89500, 2)
(89500,)
(89500,)
(89500,)
(89500,)


<ipython-input-341-6d6061c1b2ee>:13: RuntimeWarning: Mean of empty slice
  locs = np.nanmean(np.nanmean(temp_features[p*14: (p+1)*14,:,:],2),0)


In [326]:
frame_id = 855
print (len(labels_array))
clrs_new = ['blue','red','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']

# plot several frames
if False:
    fig=plt.figure()
    for t in range(40):
        ax=plt.subplot(4,10,t+1)
        means = []
        for k in range(len(labels_array[frame_id+t])):
            if len(labels_array[frame_id+t][k])>0:
                plt.scatter(np.array(labels_array[frame_id+t][k])[:,0], 
                        np.array(labels_array[frame_id+t][k])[:,1],
                        c=clrs_new[k])
            #print (np.array(labels_array[frame_id+t][k]).shape)
            if np.array(labels_array[frame_id+t][k]).shape[0]>0:
                plt.scatter(np.array(labels_array[frame_id+t][k])[:,0].mean(0),
                  np.array(labels_array[frame_id+t][k])[:,1].mean(0),
                  s=100, c=clrs_new[k])


        plt.title(str(t+frame_id)+" " +str(round((t+frame_id)/25.,1)),fontsize=8)
        plt.ylim(1024,0)
        plt.xlim(0,1280)
        plt.xticks([])
        plt.yticks([])
    
# plot single frame    
else:
    frame_id = 725
    #print (labels_array[200])
    #clrs_new = ['blue','red','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']

    frame_ids = [866, 867]
    for frame_id in frame_ids:
        fig=plt.figure()
        for k in range(len(labels_array[frame_id])):
            print (frame_id, k)
            print (labels_array[frame_id][k])
            if len(labels_array[frame_id][k])>0:
                plt.scatter(np.array(labels_array[frame_id][k])[:,0], 
                        np.array(labels_array[frame_id][k])[:,1],
                        c=clrs_new[k])
            #print (np.array(labels_array[frame_id+t][k]).shape)
            if np.array(labels_array[frame_id][k]).shape[0]>0:
                plt.scatter(np.array(labels_array[frame_id][k])[:,0].mean(0),
                  np.array(labels_array[frame_id][k])[:,1].mean(0),
                  s=100, c=clrs_new[k])
        plt.title(str(frame_id))
#     frame_ids = [583]
#     for frame_id in frame_ids:
#         for k in range(len(labels_array[frame_id])):

#             if np.array(labels_array[frame_id][k]).shape[0]>0:
#                 plt.scatter(np.array(labels_array[frame_id][k])[:,0].mean(0),
#                   np.array(labels_array[frame_id][k])[:,1].mean(0),
#                   s=100, c=clrs_new[k])


    
        #plt.title(str(t+frame_id))
        plt.ylim(1024,0)
        plt.xlim(0,1280)   
    
    
plt.show()

1000
866 0
[[908.196 171.68 ]
 [923.641 196.47 ]
 [940.581 219.547]
 [948.529 242.35 ]
 [956.459 258.291]]
866 1
[[780.033 155.646]
 [811.225 148.792]
 [789.554 157.376]
 [802.163 179.011]
 [819.688 195.982]
 [836.72  197.957]
 [859.883 188.7  ]]
866 2
[[299.467 587.552]
 [295.66  563.884]
 [273.93  580.978]
 [280.479 546.112]
 [246.335 571.192]
 [247.602 546.005]
 [226.475 525.4  ]
 [202.111 499.697]
 [196.68  466.904]
 [203.816 436.308]]
866 3
[[330.77  664.217]
 [310.23  698.506]
 [307.429 674.46 ]
 [276.02  662.19 ]
 [246.061 657.027]
 [227.478 641.802]
 [227.933 612.275]]
867 0
[[779.988 115.984]
 [818.732 140.194]
 [790.423 149.655]
 [795.804 172.648]
 [807.84  195.578]
 [820.397 213.038]
 [828.218 230.353]]
867 1
[[867.993 155.824]
 [883.923 115.945]
 [891.961 124.134]
 [916.509 148.223]
 [940.357 162.573]
 [946.985 180.829]
 [955.437 204.76 ]
 [956.533 243.877]
 [956.233 228.174]
 [908.024 219.63 ]]
867 2
[[308.138 594.649]
 [298.399 564.056]
 [275.57  573.735]
 [283.038 546.76

In [318]:
# figure out index of starting frame and move video to location
# initialize video writer for file to be saved

# Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
clrs = ['red','blue','yellow','green','cyan','pink','magenta','white','lightgreen','lightblue']

fname_original_vid = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressed.avi'
fname_original_vid = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.mp4'
# grab frame and update it
start = kmeans_start
original_vid = cv2.VideoCapture(fname_original_vid)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2



#start = 0
end = kmeans_end

fname_out = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/test.avi'
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out, fourcc, 25, (1280,1024), True)

frame_empty = np.zeros((1024,1280,3),'uint8')
for k in range(0,end-start,1):
    if k %100==0:
        print (k)
    #frame_cropped = frame_empty.copy()  #[:,150:1174][::2,::2]
    ret, frame = original_vid.read()
    #img.set_data(frame)


    # plot individual features
    #print (labels_array[k][0])
    for p in range(len(labels_array[k])):
        #print (labels_array[k])
        #for j in range(len(labels_array[k][p])):
        for i in range(len(labels_array[k][p])):
            x = labels_array[k][p][i][0]
            y = labels_array[k][p][i][1]

            if np.isnan(x) or np.isnan(y):
                continue
            x=int(x)
            y=int(y)

            frame[y-5:y+5,x-5:x+5]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

    cv2.putText(frame,str(k), 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)



    video_out.write(frame)
    #out.write(frame_cropped)
    
video_out.release()
cv2.destroyAllWindows()

0
100
200
300
400
500
600
700
800
900


In [133]:
print (labels_array[0])

[array([[1099.073,  266.517],
       [1131.736,  261.217],
       [1098.116,  232.794],
       [1147.752,  241.547],
       [1117.122,  208.204],
       [1146.195,  211.552],
       [1157.156,  196.13 ],
       [1164.34 ,  181.492],
       [1163.98 ,  171.988],
       [1155.223,  156.346],
       [1148.054,  148.231],
       [1116.247,  123.613],
       [1139.722,  124.144]]), array([[522.675, 400.744],
       [487.01 , 386.086],
       [491.001, 430.658],
       [450.119, 384.115],
       [465.061, 444.893],
       [448.482, 417.508],
       [413.803, 422.976],
       [380.347, 427.377],
       [354.359, 436.031],
       [323.858, 441.345],
       [297.671, 450.592],
       [269.017, 459.321],
       [248.355, 471.515],
       [222.005, 484.19 ]]), array([[587.905, 662.773],
       [581.203, 692.294],
       [601.412, 680.876],
       [602.202, 721.122],
       [625.232, 698.402],
       [611.561, 709.205],
       [627.785, 724.763],
       [643.075, 733.96 ],
       [652.894, 739.376

In [119]:
from sklearn.cluster import KMeans

# load data
print (traces_inferences.shape)
#[start:end,feature]
labels = []
ctr=0
kmeans_start = 50000
kmeans_end = kmeans_start+500
to_plot=True

# for each frame fit keep a histroy of previous 5 frames kmeans centres
kmeans_centres_history=np.zeros((5,4,2),'float32')
# loop over frames
for k in range(kmeans_start, kmeans_end):
    labels.append([])
    print (k)
    #
    flat_list = []
    list_idx = []
    for n, sublist in enumerate(traces_inferences[k]):
        list_idx.extend(np.zeros(len(sublist),'int32')+n)
        for item in sublist:
            flat_list.append(item)

    #print (flat_list)
    locs = np.vstack(flat_list).copy()
    list_idx = np.array(list_idx)

    if k==kmeans_start:
        kmeans = KMeans(n_clusters=4, random_state=0).fit(locs)
    else:
        temp_centres = previous_centres #+np.random.rand(previous_centres.shape[0], previous_centres.shape[1])*100
        kmeans = KMeans(n_clusters=4,init=temp_centres,n_init=1, max_iter=1).fit(locs)
        #kmeans = KMeans(n_clusters=4).fit(locs)
               
    # visualize reulsts
    temp_labels = []
    for k in range(4):
        idx = np.where(kmeans.labels_==k)[0]
        temp_labels.append(locs[idx])
        labels[ctr].append([])  # add empty lists
        
    # match the new labels with the old ones:
    if ctr>0:
        #print (kmeans.cluster_centers_)
        order = []
        #temp_previous_centres=kmeans_centres_history.copy()
        temp_previous_centres = previous_centres
        for k in range(4):
            vect = kmeans.cluster_centers_[k]-temp_previous_centres
            min_arg = np.argmin(scipy.spatial.distance.cdist(vect*0, vect))

            # match the curretn kmeans centre to the best historical match
#             centres_dists_median = []
#             for z in range(temp_previous_centres.shape[1]):
#                 vect = kmeans.cluster_centers_[k]-temp_previous_centres[:,z]
#                 #print ("Kmeans centres: ", kmeans.cluster_centers_[k])
#                 #print ("centre histories: ", kmeans_centres_history[:,z])
    
#                 median_dist = np.median(scipy.spatial.distance.cdist(vect*0, vect))
#                 centres_dists_median.append(median_dist)
                
#             #print ("animal: ", k, "centres_dists_median: ", centres_dists_median)
#             min_arg = np.argmin(centres_dists_median) 
            
            
            #min_arg = np.argmin(scipy.spatial.distance.cdist(vect*0, vect))
            labels[ctr][min_arg].append(temp_labels[k].copy())

            # replace match with large number to mimic hungarian algorithm
            #temp_previous_centres[:,min_arg] = 1E10
            temp_previous_centres[min_arg] = 1E10
            order.append(min_arg)
        
#         print ("FINAL ORDER: ", order)
#         print ("")
#         print ("")
#         print ("")
#         print ("")
        # fix cluster feature chimeras
        
        
        # FIRST CHIMERA: MERGED ANIMALS WITH MULTIPLE FEATURES PER ANIMAL
        for a in range(4):
            idx = np.where(kmeans.labels_==a)[0]
            if idx.shape[0]==0:
                print ("MISSING ANIMAL")
                print ("#################################################################")
                print ('')
                print ('')
                print ('')
                print ('')
                print ('')
                print ('')
                break
            #print (idx)
            #print ("locsL", locs.shape)
            #print ("list idx: ", list_idx)

            extra_feat_ids = np.unique(list_idx[idx], return_index=True, return_counts=True)
            # check to see if any duplicated features occur
            if np.max(extra_feat_ids[2])>1:
                print ("feature ids: ", list_idx[idx])
                #print ("ANIMAL: ", a, ' has extra feat ids: ', extra_feat_ids)
                idx3 = np.where(extra_feat_ids[2]>1)[0]
                
                print ("# of duplicated features ", idx3.shape)
                # loop over each extra duplicated feature and reassign it to nearby cluster
                for k1 in range(idx3.shape[0]):
                    # duplicate feature
                    dup_feature = extra_feat_ids[0][idx3[k1]]
                    print ("dup feature id: ", dup_feature)
                    
                    # grab locations of duplicated features 
                    idx4 = np.where(list_idx[idx]==dup_feature)[0]
                    locs_dup_features = locs[idx[idx4]]
                    
                    print ("idx dup features: ", idx4, 
                           "Locs duplicated features: ", locs_dup_features)

                    # find distances to all assigned locs and pick nearest to all other points in animal
                    locs_animal = locs[idx]
                    
                    dists= []
                    for k2 in range(locs_dup_features.shape[0]):
                        vect = locs_animal - locs_dup_features[k2]
                        dist_sums = scipy.spatial.distance.cdist(vect*0, vect).sum()
                        
                        dists.append(dist_sums)
                    
                    idx_correct = np.argmin(dists)
                    print ("dists:", dists, " idx corect: ", idx_correct, " keep : ", 
                          locs_dup_features[idx_correct])
                    
                    # find membership for the other 
                    for k3 in range(locs_dup_features.shape[0]):
                        
                        # delete the feature from the current animal
                        print ("DELETING FROM CURRENT ANIMAL")
                        #print (labels[ctr][a][0])
#                         print (list_idx[idx])
#                         print (locs[list_idx[idx]])
#                         print (labels[ctr][a])
                        
#                         print (locs_dup_features[k3])
                        idx9 = np.where(labels[ctr][a][0][:,0]==locs_dup_features[k3][0])[0]
                        print (labels[ctr][a][0][idx9])
                        labels[ctr][a][0][idx9] = np.nan
                        print (labels[ctr][a][0][idx9])

                            
                            
                            
                        
                        # compute distance between extra features and all other animals:
                        
                        dists_to_other_animals=[]
                        animal_ids = []
                        for b in range(4):
                            if b==a:
                                continue
                            idx6 = np.where(kmeans.labels_==b)[0]

                            locs_animal = locs[idx6]
                            vect = locs_animal - locs_dup_features[k3]
                            dist_sums = scipy.spatial.distance.cdist(vect*0, vect).sum()

                            dists_to_other_animals.append(dist_sums)
                            animal_ids.append(b)
                        
                        animal_ids=np.array(animal_ids)
                        nearest_animal_id = animal_ids[np.argmin(dists_to_other_animals)]                       
                        
                        print ("animal ", a, " extra feature: ", k3, "  nearest animal id: ", 
                               nearest_animal_id," dist: ", dists_to_other_animals)
                        
                        # check if other animal already has this feature, otherwise replace it:
                        idx7 = np.where(kmeans.labels_==nearest_animal_id)[0]
                        nearest_animal_features = list_idx[idx7]
                        
                        if dup_feature in nearest_animal_features:
                            print ("neareast animal has already feature : DELETING FEAUTRE")
                            
                            # delete feature from recording
                            print ("locs dup feature: ", locs_dup_features[k3])
                            print (" labels: ", labels[ctr][a])
                            idx8 = np.where(labels[ctr][a][0][:,0]==locs_dup_features[k3][0])[0]
                            print ("Idx8", idx8)
                            print ("deleting :", idx8, labels[ctr][a][0][idx8])
                            
                            print (labels[ctr][a][0][idx8])
                            labels[ctr][a][0][idx8] = np.nan
                            print (labels[ctr][a][0][idx8])
                            
                            #if to_plot==True:
                            if False:
                                fig =plt.figure()
                                #ax=plt.subplot(1,2,1)
                                plt.scatter(labels[ctr][a][0][:,0], labels[ctr][a][0][:,1])


                                #ax=plt.subplot(1,2,1)
                                plt.scatter(labels[ctr][a][0][:,0], labels[ctr][a][0][:,1])
                                plt.show()
                                to_plot=False
                            #break
  

                        #else:
                        #    labels[ctr][nearest_animal_id][0].append(locs_dup_features[k3])

            
                            #pass
                            
                            

                        
                        # delete extra feature
                        
                        
                print ("*********************************")
                print ("*********************************")
                print ("*********************************")
                print('')
                print('')
                print('')
                print('')
            #break
            
            # SECOND CHIMERA: OVERSPLIT ANIMAL

            #break
        
        #break
        
        # reorder previous centres
        order = np.array(order)
        print ("ORDER: ", order)
        previous_centres = [None]*4
        for k in range(4):
            previous_centres[order[k]]=kmeans.cluster_centers_[k]

        previous_centres = np.vstack(previous_centres)
         
        # append to a list of history centres;
#         kmeans_centres_history[1:]=kmeans_centres_history[:-1]
#         kmeans_centres_history[0] = kmeans.cluster_centers_
        
    else:
        previous_centres = kmeans.cluster_centers_.copy()
        for k in range(4):
            labels[0][k].append(temp_labels[k])
        
        kmeans_centres_history[:]=kmeans.cluster_centers_.copy()
        #print ("reordered centres: ", previous_centres)
   
    #print ('')
            
    ctr+=1
    

(89989, 14)
50000
50001
ORDER:  [0 1 2 3]
50002
feature ids:  [ 0  0  1  2  3  4  5  6  7  8  9 10 11 12 13]
# of duplicated features  (1,)
dup feature id:  0
idx dup features:  [0 1] Locs duplicated features:  [[1164.026  132.164]
 [1075.427  220.182]]
dists: [16855.949264265015, 17411.212142631914]  idx corect:  0  keep :  [1164.026  132.164]
DELETING FROM CURRENT ANIMAL
[[1164.026  132.164]]
[[nan nan]]
animal  1  extra feature:  0   nearest animal id:  3  dist:  [195520.7205741349, 139036.0333535187, 65436.001256690186]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [1164.026  132.164]
 labels:  [array([[     nan,      nan],
       [1075.427,  220.182],
       [1107.856,  228.564],
       [1083.853,  198.281],
       [1129.93 ,  217.66 ],
       [1099.741,  179.284],
       [1124.467,  189.104],
       [1146.818,  178.798],
       [1155.384,  156.196],
       [1164.126,  155.796],
       [1164.627,  140.398],
       [1172.018,  227.737],
       [1165.142,

ORDER:  [0 1 2 3]
50091
ORDER:  [0 1 2 3]
50092
ORDER:  [0 1 2 3]
50093
ORDER:  [0 1 2 3]
50094
feature ids:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 13]
# of duplicated features  (1,)
dup feature id:  13
idx dup features:  [13 14] Locs duplicated features:  [[590.284 691.869]
 [ 19.987 956.035]]
dists: [35990.901857129385, 158433.87932676676]  idx corect:  0  keep :  [590.284 691.869]
DELETING FROM CURRENT ANIMAL
[[590.284 691.869]]
[[nan nan]]
animal  2  extra feature:  0   nearest animal id:  0  dist:  [83314.16345021105, 104864.20495602081, 98719.55361706468]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [590.284 691.869]
 labels:  [array([[801.913, 609.98 ],
       [777.222, 607.226],
       [794.779, 635.156],
       [758.656, 620.631],
       [762.985, 652.925],
       [753.561, 640.344],
       [726.603, 644.347],
       [701.359, 644.858],
       [684.408, 648.033],
       [667.181, 652.158],
       [648.35 , 659.347],
       [631.184, 669.161],


animal  0  extra feature:  0   nearest animal id:  3  dist:  [87381.78717583997, 125857.67158963223, 71481.4755877555]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [420.743  75.312]
 labels:  [array([[546.606, 412.071],
       [554.801, 387.183],
       [515.407, 402.432],
       [562.849, 364.007],
       [499.494, 374.941],
       [523.757, 354.993],
       [514.711, 317.127],
       [498.286, 278.048],
       [500.632, 236.748],
       [517.667, 204.691],
       [    nan,     nan],
       [    nan,     nan],
       [    nan,     nan],
       [530.866, 147.2  ],
       [490.971, 124.898],
       [453.759,  99.626]])]
Idx8 []
deleting : [] []
[]
[]
DELETING FROM CURRENT ANIMAL
[[530.866 147.2  ]]
[[nan nan]]
animal  0  extra feature:  1   nearest animal id:  3  dist:  [72997.70319206206, 104194.01763583967, 51478.842294008966]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [530.866 147.2  ]
 labels:  [array([[546.606, 412.071],
 

[[nan nan]]
animal  0  extra feature:  1   nearest animal id:  2  dist:  [168918.04021912755, 67291.55308427966, 109123.49308570538]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [229.192 626.954]
 labels:  [array([[283.821, 572.027],
       [252.482, 620.104],
       [251.809, 619.616],
       [220.074, 619.981],
       [212.254, 612.234],
       [211.513, 587.46 ],
       [211.091, 549.107],
       [    nan,     nan],
       [    nan,     nan],
       [    nan,     nan],
       [    nan,     nan],
       [257.647, 446.882],
       [262.801, 418.533],
       [267.256, 393.234],
       [261.214, 363.965]])]
Idx8 []
deleting : [] []
[]
[]
*********************************
*********************************
*********************************




feature ids:  [ 0  1  2  3  4  5  6  7 10 11 12 13 13]
# of duplicated features  (1,)
dup feature id:  13
idx dup features:  [11 12] Locs duplicated features:  [[1170.541  205.039]
 [1174.122  252.47 ]]
dists: [8836.1674

ORDER:  [0 1 2 3]
50223
ORDER:  [0 1 2 3]
50224
ORDER:  [0 1 2 3]
50225
ORDER:  [0 1 2 3]
50226
ORDER:  [0 1 2 3]
50227
ORDER:  [0 1 2 3]
50228
ORDER:  [0 1 2 3]
50229
ORDER:  [0 1 2 3]
50230
ORDER:  [0 1 2 3]
50231
ORDER:  [0 1 2 3]
50232
ORDER:  [0 1 2 3]
50233
ORDER:  [0 1 2 3]
50234
feature ids:  [0 0 3 3 4 4 5 5 6 7 8 9]
# of duplicated features  (4,)
dup feature id:  0
idx dup features:  [0 1] Locs duplicated features:  [[683.395 563.839]
 [643.765 756.066]]
dists: [15469.377780325165, 13406.66107254021]  idx corect:  1  keep :  [643.765 756.066]
DELETING FROM CURRENT ANIMAL
[[683.395 563.839]]
[[nan nan]]
animal  2  extra feature:  0   nearest animal id:  3  dist:  [76760.15675704058, 84188.05279634803, 40965.867273532145]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [683.395 563.839]
 labels:  [array([[    nan,     nan],
       [643.765, 756.066],
       [675.892, 619.91 ],
       [677.398, 739.903],
       [691.981, 619.874],
       [646.006, 729.6

       [1179.657,  251.13 ]])]
Idx8 []
deleting : [] []
[]
[]
DELETING FROM CURRENT ANIMAL
[[1159.654  227.759]]
[[nan nan]]
animal  1  extra feature:  1   nearest animal id:  3  dist:  [119856.51472320699, 84637.81686950203, 34383.93245375716]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [1159.654  227.759]
 labels:  [array([[1124.658,  260.255],
       [1147.524,  241.502],
       [1115.706,  229.668],
       [     nan,      nan],
       [     nan,      nan],
       [1122.   ,  205.481],
       [1147.441,  198.55 ],
       [1146.061,  179.05 ],
       [1148.349,  148.762],
       [1148.092,  131.859],
       [1172.706,  251.552],
       [1180.538,  251.967],
       [1179.657,  251.13 ]])]
Idx8 []
deleting : [] []
[]
[]
*********************************
*********************************
*********************************




ORDER:  [0 1 2 3]
50279
feature ids:  [ 0  1  2  3  3  4  5  6  7  8  9 11 12 13]
# of duplicated features  (1,)
dup feature id:  3
id

animal  2  extra feature:  0   nearest animal id:  1  dist:  [63903.57371187871, 61513.30223089123, 70416.81481863886]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [572.162 707.071]
 labels:  [array([[    nan,     nan],
       [555.514, 808.211],
       [565.072, 780.223],
       [532.916, 787.223],
       [571.764, 754.767],
       [525.22 , 757.128],
       [547.963, 756.426],
       [554.955, 731.987],
       [562.242, 709.172],
       [571.312, 699.425],
       [581.878, 685.562],
       [595.66 , 663.746],
       [608.711, 651.896],
       [626.807, 645.256],
       [637.059, 644.105]])]
Idx8 []
deleting : [] []
[]
[]
DELETING FROM CURRENT ANIMAL
[[555.514 808.211]]
[[nan nan]]
animal  2  extra feature:  1   nearest animal id:  1  dist:  [69719.86765848435, 68067.51114538783, 82414.69917810906]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [555.514 808.211]
 labels:  [array([[    nan,     nan],
       [    nan,     nan],
   

[[nan nan]]
animal  2  extra feature:  1   nearest animal id:  0  dist:  [42088.64611841702, 54243.73097656746, 68248.80107961613]
*********************************
*********************************
*********************************




ORDER:  [0 1 2 3]
50398
feature ids:  [ 0  0  1  2  3  4  5  6  7  8  9 10 11]
# of duplicated features  (1,)
dup feature id:  0
idx dup features:  [0 1] Locs duplicated features:  [[442.615 708.879]
 [532.63  795.095]]
dists: [19661.07508607196, 13607.995172855735]  idx corect:  1  keep :  [532.63  795.095]
DELETING FROM CURRENT ANIMAL
[[442.615 708.879]]
[[nan nan]]
animal  2  extra feature:  0   nearest animal id:  0  dist:  [22232.00984353079, 55051.745595205735, 65339.46529032451]
DELETING FROM CURRENT ANIMAL
[[532.63  795.095]]
[[nan nan]]
animal  2  extra feature:  1   nearest animal id:  0  dist:  [42161.10170894428, 54076.06853311474, 68261.89179752798]
*********************************
*********************************
************************

       [1116.301,  124.593]])]
Idx8 []
deleting : [] []
[]
[]
DELETING FROM CURRENT ANIMAL
[[1161.596  259.652]]
[[nan nan]]
animal  1  extra feature:  1   nearest animal id:  3  dist:  [184431.45381499743, 92878.88258399721, 28575.390730658615]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [1161.596  259.652]
 labels:  [array([[     nan,      nan],
       [     nan,      nan],
       [1138.998,  272.128],
       [1112.942,  237.683],
       [     nan,      nan],
       [     nan,      nan],
       [1130.803,  221.373],
       [1163.117,  226.468],
       [1163.096,  212.367],
       [1163.699,  188.392],
       [1171.381,  188.083],
       [1171.612,  180.423],
       [1154.198,  164.424],
       [1140.679,  147.457],
       [1122.961,  132.705],
       [1116.301,  124.593]])]
Idx8 []
deleting : [] []
[]
[]
*********************************
*********************************
*********************************




ORDER:  [0 1 2 3]
50445
feature ids:  [ 0  0  

feature ids:  [ 0  1  2  3  4  5  5  6  7  8  9 10 11 12 13]
# of duplicated features  (1,)
dup feature id:  5
idx dup features:  [5 6] Locs duplicated features:  [[406.264 428.017]
 [299.804 459.65 ]]
dists: [20991.465299010506, 20512.3270663551]  idx corect:  1  keep :  [299.804 459.65 ]
DELETING FROM CURRENT ANIMAL
[[406.264 428.017]]
[[nan nan]]
animal  0  extra feature:  0   nearest animal id:  2  dist:  [129696.0779359105, 42821.3702892084, 48217.95019685404]
neareast animal has already feature : DELETING FEAUTRE
locs dup feature:  [406.264 428.017]
 labels:  [array([[482.767, 405.91 ],
       [445.805, 398.218],
       [455.45 , 438.103],
       [412.977, 397.005],
       [427.354, 457.211],
       [    nan,     nan],
       [299.804, 459.65 ],
       [387.922, 449.261],
       [349.101, 450.901],
       [316.052, 453.126],
       [285.13 , 466.987],
       [262.982, 492.456],
       [251.692, 508.384],
       [244.1  , 532.265],
       [243.902, 555.836]])]
Idx8 []
deleting : [

In [6]:
print(len(labels))
#print (labels[0])

for k in range(100):
    ax=plt.subplot(10,10,k+1)
    for p in range(4):
        print (labels[k][p][0])
        plt.scatter(labels[k][p][0][:,0], labels[k][p][0][:,1])
plt.show()




100
[[794.264 722.01 ]
 [816.495 731.503]
 [840.578 740.647]
 [864.179 745.718]
 [892.331 747.273]]
[[212.008 578.765]
 [195.583 596.464]
 [224.168 592.936]
 [202.058 627.663]
 [244.894 699.448]
 [231.548 616.887]
 [219.762 635.786]
 [229.313 652.455]
 [290.371 667.141]
 [308.512 652.325]
 [324.795 644.912]]
[[682.948 645.044]
 [676.196 667.274]
 [694.192 666.452]
 [693.248 706.273]
 [712.616 683.532]
 [708.462 705.227]
 [725.17  709.022]
 [749.431 715.548]
 [770.028 715.206]]
[[468.264 723.505]
 [467.456 707.898]
 [429.01  691.943]
 [434.513 724.036]
 [422.42  709.587]
 [403.24  708.016]
 [387.677 699.793]
 [371.791 683.972]
 [315.73  731.703]
 [316.309 732.257]
 [339.703 717.328]]
[[793.666 722.164]
 [815.656 731.223]
 [840.005 740.615]
 [863.436 746.047]
 [892.312 747.475]]
[[212.274 571.664]
 [195.574 595.89 ]
 [224.989 592.648]
 [197.627 626.435]
 [244.85  699.416]
 [232.172 616.563]
 [219.758 635.75 ]
 [228.328 652.85 ]
 [284.354 661.249]
 [308.027 652.23 ]
 [323.955 645.072]]
[[

[[790.454 715.197]
 [811.471 727.877]
 [836.898 740.866]
 [859.24  745.715]
 [883.844 748.284]]
[[180.48  635.838]
 [259.748 708.295]
 [188.575 651.72 ]
 [196.815 620.124]
 [209.865 667.895]
 [223.994 625.787]
 [219.306 644.735]
 [252.946 652.077]
 [289.51  646.927]
 [322.537 643.031]
 [330.801 628.577]]
[[676.94  653.257]
 [676.735 667.611]
 [696.184 665.356]
 [690.273 699.854]
 [711.624 676.236]
 [705.73  699.328]
 [722.154 707.574]
 [740.881 712.056]
 [763.733 714.932]]
[[500.228 757.769]
 [499.858 740.079]
 [476.053 762.627]
 [484.171 723.761]
 [451.3   731.646]
 [466.721 723.234]
 [444.23  709.08 ]
 [429.051 700.887]
 [412.13  690.423]
 [402.206 692.362]
 [387.277 692.725]
 [372.053 707.315]]
[[792.486 715.974]
 [811.557 728.027]
 [836.934 740.82 ]
 [859.36  745.991]
 [884.417 748.285]]
[[180.431 635.844]
 [188.503 651.491]
 [196.9   620.21 ]
 [210.812 667.916]
 [218.732 620.95 ]
 [226.725 643.997]
 [252.783 651.743]
 [289.082 646.025]
 [315.985 643.518]
 [330.903 628.396]]
[[677.

[[213.635 572.397]
 [212.079 595.728]
 [223.878 592.52 ]
 [202.982 634.227]
 [233.039 614.882]
 [220.093 635.662]
 [244.435 651.432]
 [293.617 651.343]
 [323.655 643.857]
 [227.821 699.634]
 [331.011 628.313]
 [267.866 724.676]
 [299.406 731.583]]
[[682.95  653.866]
 [676.583 667.501]
 [697.713 664.329]
 [682.693 704.939]
 [711.86  679.593]
 [700.494 700.31 ]
 [720.722 709.485]
 [741.837 715.008]
 [763.781 715.442]]
[[518.779 747.187]
 [507.856 732.017]
 [493.37  747.667]
 [492.723 715.6  ]
 [467.097 731.048]
 [468.011 708.989]
 [451.589 707.381]
 [422.135 699.044]
 [411.33  695.499]
 [395.099 697.091]
 [379.143 691.723]
 [362.842 693.122]
 [341.381 715.608]
 [340.114 715.716]]
[[793.359 715.801]
 [812.211 726.116]
 [838.253 739.611]
 [859.982 745.323]
 [892.062 747.358]]
[[218.114 578.149]
 [211.91  595.833]
 [222.427 592.911]
 [202.665 634.262]
 [232.227 614.954]
 [219.983 635.784]
 [244.523 651.322]
 [292.481 659.504]
 [323.464 636.203]
 [244.061 707.676]
 [331.562 636.304]
 [275.81

[[792.808 716.071]
 [811.901 727.353]
 [837.14  739.296]
 [859.719 744.946]
 [884.148 747.612]]
[[218.653 565.381]
 [195.694 595.477]
 [226.113 586.439]
 [202.801 628.227]
 [233.414 613.877]
 [219.203 635.281]
 [244.66  651.385]
 [258.876 667.694]
 [315.749 644.326]
 [242.836 700.353]
 [261.323 722.683]
 [293.181 731.399]
 [332.226 716.106]]
[[678.785 652.771]
 [675.983 668.502]
 [697.962 659.711]
 [691.114 698.783]
 [711.219 673.603]
 [706.22  698.976]
 [723.762 708.813]
 [742.071 714.515]
 [763.455 715.372]]
[[523.979 724.409]
 [516.233 723.723]
 [514.778 747.671]
 [491.884 716.251]
 [484.12  741.613]
 [470.17  731.866]
 [451.375 721.701]
 [428.749 707.992]
 [412.611 699.718]
 [399.639 696.786]
 [384.84  692.111]
 [364.169 693.618]
 [341.189 714.628]]
[[793.09  716.176]
 [812.127 727.427]
 [837.382 739.238]
 [859.825 745.17 ]
 [884.251 747.829]]
[[213.929 565.337]
 [196.212 595.223]
 [225.957 586.394]
 [202.952 629.497]
 [233.087 614.545]
 [219.505 635.361]
 [244.517 651.429]
 [275.2

[[683.955 645.545]
 [676.159 668.598]
 [702.143 660.774]
 [691.501 698.961]
 [714.152 675.713]
 [708.128 699.043]
 [724.077 708.762]
 [741.743 715.031]
 [759.777 715.743]
 [675.983 676.018]]
[[532.716 731.597]
 [516.525 723.459]
 [514.738 747.465]
 [491.947 716.707]
 [485.618 747.517]
 [468.793 731.943]
 [451.236 715.208]
 [428.424 708.001]
 [416.313 699.574]
 [401.378 696.494]
 [386.825 692.315]
 [364.232 693.985]]
[[791.622 716.508]
 [812.242 730.103]
 [836.09  739.919]
 [859.776 745.934]
 [884.466 748.104]]
[[218.931 565.84 ]
 [196.54  594.505]
 [226.92  585.209]
 [202.849 627.285]
 [233.992 614.244]
 [218.828 628.885]
 [244.397 651.34 ]
 [291.428 659.57 ]
 [323.39  652.023]
 [228.357 692.125]
 [331.619 643.883]
 [283.975 715.978]
 [299.568 732.115]
 [340.918 715.095]
 [323.155 716.314]]
[[683.602 643.302]
 [676.299 669.277]
 [698.42  655.01 ]
 [690.841 694.047]
 [715.496 673.31 ]
 [708.917 692.971]
 [725.616 706.447]
 [741.606 712.329]
 [763.494 715.352]]
[[532.744 731.477]
 [516.1

[[674.491 644.249]
 [683.325 707.122]
 [659.027 700.539]
 [707.986 643.638]
 [667.696 726.658]
 [698.462 685.023]
 [726.652 660.169]
 [713.63  677.405]
 [726.801 694.48 ]
 [746.518 709.871]
 [765.347 714.143]]
[[633.54  703.105]
 [636.011 739.538]
 [627.238 723.959]
 [598.889 720.413]
 [576.353 716.344]
 [562.956 707.794]
 [563.761 692.116]]
[[791.204 719.462]
 [812.675 730.078]
 [836.326 742.383]
 [859.606 746.895]
 [884.517 748.75 ]]
[[212.174 566.052]
 [196.041 595.866]
 [226.652 586.036]
 [202.866 628.139]
 [    nan     nan]
 [219.576 635.431]
 [244.409 651.209]
 [274.625 661.871]
 [315.736 659.767]
 [228.149 707.902]
 [339.426 676.032]
 [275.476 723.626]
 [356.056 684.688]
 [294.235 731.124]
 [354.932 705.606]
 [346.296 716.381]
 [364.107 691.164]]
[[667.109 643.49 ]
 [683.285 706.946]
 [659.078 700.397]
 [700.402 645.199]
 [667.794 726.648]
 [698.263 684.9  ]
 [727.2   660.016]
 [713.267 678.196]
 [727.066 694.241]
 [745.959 709.735]
 [765.269 714.299]]
[[633.658 703.252]
 [639.3

In [86]:
# figure out index of starting frame and move video to location
# initialize video writer for file to be saved

# Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
clrs = ['red','blue','yellow','green']


fname_original_vid = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressed.avi'
# grab frame and update it
start = kmeans_start
original_vid = cv2.VideoCapture(fname_original_vid)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    
    
#start = 0
end = kmeans_end

fname_out = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/test.avi'
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out, fourcc, 25, (1280,1024), True)

frame_empty = np.zeros((1024,1280,3),'uint8')
for k in range(0,end-start,1):
    if k %100==0:
        print (k)
    #frame_cropped = frame_empty.copy()  #[:,150:1174][::2,::2]
    ret, frame = original_vid.read()
    #img.set_data(frame)


    # plot individual features
    for p in range(len(labels[k])):
        for j in range(len(labels[k][p][0])):
            for i in range(labels[k][p][0].shape[0]):
                x = labels[k][p][0][i,0]
                y = labels[k][p][0][i,1]
            
                if np.isnan(x) or np.isnan(y):
                    continue
                x=int(x)
                y=int(y)
            
                frame[y-5:y+5,x-5:x+5]= (np.float32(
                    matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')
    
    video_out.write(frame)
    #out.write(frame_cropped)
    
video_out.release()
cv2.destroyAllWindows()


0
100
200
300
400


#
#
# OTHER FUNCTIONS FOR LABELING VIDEOS ETC

#
#

In [ ]:
# LOAD VIDE TO ANNOTATE
video_name = "/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi"
frame_rate = 25


In [ ]:
# video writing step
#video_out.release()
#out.release()
#original_vid.release()
#cv2.destroyAllWindows()

# Robust outlier detection;
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

       
original_vid = cv2.VideoCapture(video_name)
#cap = cv2.VideoCapture('chaplin.mp4')

ctr=0
# figure out index of starting frame and move video to location
frame_no1 = 0 #56*60*frame_rate
original_vid.set(1,frame_no1)

# initialize video writer for file to be saved
out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/vids_out/'


# Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
n_sec = 180
clrs = ['blue','red','yellow','green']

# 
#for k in range(tracesx.shape[1]):    
#for k in range(tracesx.shape[1])[:500]:    
ctr=0
start = 7*60*25
end = start + 120*25

fname_out = out_dir+'video_labeled_'+str(start)+"_"+str(end)+'.avi'
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
# fourcc = cv2.VideoWriter_fourcc(*'MP4V')
# fourCc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (1280,1024), True)


start_frame_number = start
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)

for k in range(start,end,1):    
    if k %100==0:
        print (k)
    
    #while True:
    ret, frame = original_vid.read()
    if ret==False:
        break

#         if ctr==k:
#             ctr+=1
#             break
#         ctr+=1
            
    #frame_cropped = np.flipud(frame.copy())#[:,150:1174][::2,::2]
    frame_cropped = frame.copy()  #[:,150:1174][::2,::2]

    # PLOT EVERY FEATURE
    threshold = 0.95
    if False:
        for p in range(tracesx.shape[0]):
            l = likelihoods[p,k]
            if l < threshold:
                continue

            x = np.int32(tracesx[p,k])

            y = np.int32(tracesy[p,k])


            # plot individual features
            frame_cropped[y-5:y+5,x-5:x+5]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p//14]))*255.).astype('uint8')
    
    # PLOT MEAN FEATURE
    for p in range(0, tracesx.shape[0], 14):
        l = likelihoods[p:p+14,k]
        idx = np.where(l<threshold)[0]

        traces_local = tracesx[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 1.)
        x = np.int32(np.nanmedian(traces_local))
        
        traces_local = tracesy[p:p+14,k]
        traces_local = np.delete(traces_local, idx)
        traces_local = reject_outliers(traces_local, m = 1.)
        y = np.int32(np.nanmedian(traces_local))
        
        # plot large bump
        frame_cropped[y-15:y+15,x-15:x+15]= (np.float32(
            matplotlib.colors.to_rgb(clrs[p//14]))*255.).astype('uint8')
    

    #cv2.imshow('frame',frame_cropped)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #   break

    video_out.write(frame_cropped)
    #out.write(frame_cropped)
    
video_out.release()
original_vid.release()
cv2.destroyAllWindows()

#break
    

In [ ]:
# use graphs to connect
print (traces_inferences.shape)

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

    
#[start:end,feature]
labels = []
kmeans_start = 54000
kmeans_end = kmeans_start+1000

max_dist = 100
min_network = 4
min_dist_chunks = 300

labels_array = []
ctr=0
# loop over frames
for k in range(kmeans_start, kmeans_end):
    print (k)

    # make label array
    labels_array.append([])

    #
    flat_list = []
    list_idx = []
    for n, sublist in enumerate(traces_inferences[k]):
        list_idx.extend(np.zeros(len(sublist),'int32')+n)
        for item in sublist:
            flat_list.append(item)

    #print (flat_list)
    locs = np.vstack(flat_list).copy()
    list_idx = np.array(list_idx)
    
    # loop over features and build graphs:
    # find nearest next feature to the selected feature and connect them:
    cc = []
    cc2 = np.zeros((list_idx.shape[0],list_idx.shape[0]),'int32')
    for feature in range(np.unique(list_idx).shape[0]-1):
        idx = np.where(list_idx==feature)[0]
        #print (feature, idx)
        
        locs_feat = locs[idx]
        #print ("locs feat: ", locs_feat)
    
        idx_next = np.where(list_idx==feature+1)[0]
        locs_next = locs[idx_next]
        #print (locs_next)
        if locs_next.shape[0]==0:
            continue
        
        for p in range(locs_feat.shape[0]):
            loc_ = locs_feat[p]
            #print ("loc_:", loc_)
            
            vect = locs_next-loc_
            min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
            
            if min_dist<max_dist:
                argmin = np.argmin(scipy.spatial.distance.cdist(vect*0, vect))
                #print (argmin, idx_next)
                cc.append([idx[p], idx_next[argmin]])
                cc2[idx[p], idx_next[argmin]]=1
                
    graph=np.array(cc2)
    graph = csr_matrix(graph)
    #print (graph)
    #print (graph.shape)
    
    n_components, labels_all = connected_components(csgraph=graph, directed=False, return_labels=True)
    #print (n_components)
    
    # keep only top 6 chunks of networks
    labels_unique = np.unique(labels_all, return_counts=True)
    idx_nets = np.where(labels_unique[1]>min_network)[0]
    print ("idx_nets:", idx_nets)
    labels_kept = labels_unique[0][idx_nets]
    
    #print ("LABELS post cc: ", labels)

    # save previous centres to match them and overwrite below
    # loop over all network components
    if ctr>5:
        # make large list to hold ftuure labels
        for p in range(20):
            labels_array[ctr].append([])
        
        # keep track of the matches with previous chunks
        idx_matches = []
#         previous_locs = labels_array[ctr-1].copy()
#         previous_centres = []
#         for q in range(len(previous_locs)):
#             if len(previous_locs[q])>0:
#                 previous_centres.append(np.mean(previous_locs[q],0))
         
        #if ctr>5:
        #previous_locs = labels_array[ctr-1].copy()
        previous_centres_array = []
        for qq in range(5):
            previous_locs = labels_array[ctr-1-qq].copy()
            previous_centres_array.append([])
            for q in range(len(previous_locs)):
                if len(previous_locs[q])>0:
                    previous_centres_array[qq].append(np.mean(previous_locs[q],0))

        
        previous_centres = np.array(previous_centres)
        print ("previous cnetres: ", previous_centres.shape)
        #break
        
        for p in np.unique(labels_kept):
            idx = np.where(labels_all==p)[0]

            locs_network = locs[idx]
            centre_network = np.mean(locs_network,0)
            print ("centre network", p, centre_network)

            # search for current network chunk for nearest previous chunk
            # make a list of distances
            min_dists = []
            for zz in range(5):
                temp_dists=[]
                for z in range(len(previous_centres_array[zz])):
                    if len(previous_centres_array[zz][z])==0:
                        #break
                        min_dists.append(1E10)
                    else:
                        #print (k,p,z,"labels array previous [z]: ", previous_centres[z])
                        vect = previous_centres_array[zz][z]-centre_network
                        #print ("vect: ", vect)
                        #min_dist = np.argmin(scipy.spatial.distance.cdist(vect*0, vect))
                        min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                        #min_dist = np.min(np.linalg.norm(vect))
                        #print ("min dist: ", min_dist)
                        temp_dists.append(min_dist)
                
                min_dists.append(np.min(temp_dists))
                
            print ("min dists:", min_dists)
            argmin = np.argmin(min_dists)
            if min_dists[argmin]<min_dist_chunks:
                print ("Matched to previous centre: ", argmin)
                #print (labels_array)
                #previous_centres[argmin]=[1E10,1E10]
                print ("Updated previous cntres: ", previous_centres)
                # replace locations of previous match 
                idx_matches.append(argmin)
                #labels_array[ctr][argmin].append(locs[idx])
            
            # add it as a new tracklet
            else:
                # keep track of unmatched data
                #unmatched_locs[ctr_unmatched].append(locs[idx][0])
                #ctr_unmatched+=1       
                idx_matches.append(None)
            
        # write the labels for matched data
        #print ("idx matches: ", idx_matches)
        for p in range(len(idx_matches)):
            idx = np.where(labels_all==p)[0]
            print ("idx_matches[p]:", idx_matches[p])
            if idx_matches[p] is not None:
                locs_network = locs[idx]
                labels_array[ctr][idx_matches[p]]=locs_network
                print ("inserted: ", locs_network)
                print (" at location: ", idx_matches[p])
                
        # write labels for un matched data
        ctr_inner=0
        for p in range(len(idx_matches)):
            print ("reordering:", p)
            idx = np.where(labels_all==p)[0]
            if idx_matches[p] is None:
                locs_network = locs[idx]
                
                # search for an empty list to populate
                while True:
                    #print ("labels array: ", labels_array[ctr])
                    if len(labels_array[ctr][ctr_inner])==0:
                        labels_array[ctr][ctr_inner]=locs_network
                        ctr_inner+=1
                        break
                    ctr_inner+=1
                    
            
#             # After all mathcing is done, rearange the data to fit in the first 6 labels 
#             ctr_refill=0
#             for q in range(12):
#                 print ("labels array ctr: ", np.array(labels_array[ctr]).shape)
#                 if len(labels_array[ctr][q])==0:
#                     if len(unmatched_locs[ctr_refill])==0:
#                         break
#                     labels_array[ctr][q]=unmatched_locs[ctr_refill]
#                     ctr_refill+=1
        print (k, " saved labels array: ", labels_array[ctr])

    # for first time point
    else:
        for p in np.unique(labels_kept):
            idx = np.where(labels_all==p)[0]
            locs_network = locs[idx]
            #print ("locs network", locs_network)
            labels_array[ctr].append(locs_network)

        print ("FIRST saved labels array: ", labels_array)
#             break
#         break
#     break

    ctr+=1   
    print ("")
    print ("")
    
    